# **Stable Diffusion - LoRA**

# I. Installation

In [3]:
# @title ## 1.1. Install Dependencies
# @markdown Clone Kohya Trainer from GitHub and check for updates. Use textbox below if you want to checkout other branch or old commit. Leave it empty to stay the HEAD on main.  This will also install the required libraries.
import os
import zipfile
import shutil
import time
from subprocess import getoutput
from IPython.utils import capture
from run_training import train_model

# Model Training 



In [4]:
# @title ## Model Config
# Change the key to:
# - living
# - bedroom
# - dining
import json

with open('./train/LoRA/config/config.json') as json_file:
    configs = json.load(json_file)

In [5]:
#@title ## Start Training
# Replace invalid config params
configs['stop_text_encoder_training_pct'] = configs.pop('stop_text_encoder_training')

train_model(headless=True, print_only=False, **configs)

01:29:21-737414 INFO     Start training LoRA Standard ...

01:29:21-741426 INFO     Checking for duplicate image filenames in training data directory...

01:29:21-744944 INFO     Valid image folder names found in:                                                        
                         C:/Users/JackChua/Projects/NUS/SEM02/PRS-PM-2023-07-01-GRP5-roomifAI/SystemCode/src/main/t
                         rain/LoRA/train_data

01:29:21-746944 INFO     Headless mode, skipping verification if model already exist... if model already exist it  
                         will be overwritten...

01:29:21-750067 INFO     Folder 10_living: 160 images found

...

01:29:21-760170 INFO     max_train_steps (1600 / 1 / 4 * 1 * 1) = 400

01:29:21-761169 INFO     stop_text_encoder_training = 0

01:29:21-762169 INFO     lr_warmup_steps = 40

01:29:21-764169 INFO     Saving training config to                                                                 
                         C:/Users/JackChua/Projects/NUS/SEM02/PRS-PM-2023-07-01-GRP5-roomifAI/SystemCode/src/main/t
                         rain/LoRA/output\roomifai_living_20231012-012921.json...

01:29:21-766172 INFO     accelerate launch --num_cpu_threads_per_process=2 "./train_network.py" --enable_bucket    
                         --min_bucket_reso=256 --max_bucket_reso=2048                                              
                         --pretrained_model_name_or_path="runwayml/stable-diffusion-v1-5"                          
                         --train_data_dir="C:/Users/JackChua/Projects/NUS/SEM02/PRS-PM-2023-07-01-GRP5-roomifAI/Sys
                         temCode/src/main/train/LoRA/train_data" --resolution="512,512"                            
                         --output_dir="C:/Users/JackChua/Projects/NUS/SEM02/PRS-PM-2023-07-01-GRP5-roomifAI/SystemC
                         ode/src/main/train/LoRA/output"                                                           
                         --logging_dir="C:/Users/JackChua/Projects/NUS/SEM02/PRS-PM-2023-07-01-GRP5-roomifAI/System
                         Code/src/main/train/LoRA/logs" --network_alpha="1" --save_model_as=safetensors            
                         --network_module=networks.lora --text_encoder_lr=5e-05 --unet_lr=0.0001 --network_dim=8   
                         --gradient_accumulation_steps=4 --output_name="roomifai_living"                           
                         --lr_scheduler_num_cycles="1" --no_half_vae --learning_rate="0.0001"                      
                         --lr_scheduler="cosine" --lr_warmup_steps="40" --train_batch_size="1"                     
                         --max_train_steps="400" --save_every_n_epochs="1" --mixed_precision="fp16"                
                         --save_precision="fp16" --caption_extension=".txt" --cache_latents                        
                         --optimizer_type="AdamW8bit" --max_data_loader_n_workers="0" --bucket_reso_steps=64       
                         --save_every_n_steps="100" --bucket_no_upscale --noise_offset=0.0

In [10]:
# @title ## 6.2. Interrogating LoRA Weights
# @markdown Now you can check if your LoRA trained properly.
import os
import torch
import json
from safetensors.torch import load_file
from safetensors.torch import safe_open

# @markdown If you used `clip_skip = 2` during training, the values of `lora_te_text_model_encoder_layers_11_*` will be `0.0`, this is normal. These layers are not trained at this value of `Clip Skip`.
network_weight = "./LoRA/roomifai_living.safetensors"
verbose = False

def is_safetensors(path):
    return os.path.splitext(path)[1].lower() == ".safetensors"

def load_weight_data(file_path):
    if is_safetensors(file_path):
        return load_file(file_path)
    else:
        return torch.load(file_path, map_location="cuda")

def extract_lora_weights(weight_data):
    lora_weights = [
        (key, weight_data[key])
        for key in weight_data.keys()
        if "lora_up" in key or "lora_down" in key
    ]
    return lora_weights

def print_lora_weight_stats(lora_weights):
    print(f"Number of LoRA modules: {len(lora_weights)}")

    for key, value in lora_weights:
        value = value.to(torch.float32)
        print(f"{key}, {torch.mean(torch.abs(value))}, {torch.min(torch.abs(value))}")

def print_metadata(file_path):
    if is_safetensors(file_path):
        with safe_open(file_path, framework="pt") as f:
            metadata = f.metadata()
        if metadata is not None:
            print(f"\nLoad metadata for: {file_path}")
            print(json.dumps(metadata, indent=4))
    else:
        print("No metadata saved, your model is not in safetensors format")

def main(file_path, verbose: bool):
    weight_data = load_weight_data(file_path)

    if verbose:
        lora_weights = extract_lora_weights(weight_data)
        print_lora_weight_stats(lora_weights)

    print_metadata(file_path)

if __name__ == "__main__":
    main(network_weight, verbose)


Load metadata for: ./LoRA/roomifai_living.safetensors
{
    "modelspec.architecture": "stable-diffusion-v1/lora",
    "modelspec.date": "2023-10-11T01:19:58",
    "modelspec.implementation": "diffusers",
    "modelspec.prediction_type": "epsilon",
    "modelspec.resolution": "512x512",
    "modelspec.sai_model_spec": "1.0.0",
    "modelspec.title": "roomifai_living",
    "ss_adaptive_noise_scale": "None",
    "ss_base_model_version": "sd_v1",
    "ss_batch_size_per_device": "1",
    "ss_bucket_info": "{\"buckets\": {\"0\": {\"resolution\": [512, 512], \"count\": 1600}}, \"mean_img_ar_error\": 0.0}",
    "ss_bucket_no_upscale": "True",
    "ss_cache_latents": "True",
    "ss_caption_dropout_every_n_epochs": "0",
    "ss_caption_dropout_rate": "0.0",
    "ss_caption_tag_dropout_rate": "0.0",
    "ss_clip_skip": "None",
    "ss_color_aug": "False",
    "ss_dataset_dirs": "{\"10_roomifai\": {\"n_repeats\": 10, \"img_count\": 160}}",
    "ss_enable_bucket": "True",
    "ss_epoch": "4",
   

c:\Users\JackChua\miniconda3\envs\room\lib\site-packages\safetensors\torch.py:98: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  with safe_open(filename, framework="pt", device=device) as f:


In [ ]:
# @title ## 6.3. Inference
%store -r

# @markdown ### LoRA Config
# @markdown Currently, `LoHa` and `LoCon_Lycoris` are not supported. Please run `Portable Web UI` instead
network_weight = ""
network_mul = 0.7  # @param {type:"slider", min:-1, max:2, step:0.05}
network_module = "networks.lora"
network_args = ""

# @markdown ### <br> General Config
v2 = False
v_parameterization = False
prompt = "masterpiece, best quality, 1girl, aqua eyes, baseball cap, blonde hair, closed mouth, earrings, green background, hat, hoop earrings, jewelry, looking at viewer, shirt, short hair, simple background, solo, upper body, yellow shirt"  # @param {type: "string"}
negative = "lowres, bad anatomy, bad hands, text, error, missing fingers, extra digit, fewer digits, cropped, worst quality, low quality, normal quality, jpeg artifacts, signature, watermark, username, blurry"  # @param {type: "string"}
model = "/content/pretrained_model/AnyLoRA.safetensors"  # @param {type: "string"}
vae = ""  # @param {type: "string"}
outdir = "/content/tmp"  # @param {type: "string"}
scale = 7  # @param {type: "slider", min: 1, max: 40}
sampler = "ddim"  # @param ["ddim", "pndm", "lms", "euler", "euler_a", "heun", "dpm_2", "dpm_2_a", "dpmsolver","dpmsolver++", "dpmsingle", "k_lms", "k_euler", "k_euler_a", "k_dpm_2", "k_dpm_2_a"]
steps = 28  # @param {type: "slider", min: 1, max: 100}
precision = "fp16"  # @param ["fp16", "bf16"] {allow-input: false}
width = 512  # @param {type: "integer"}
height = 768  # @param {type: "integer"}
images_per_prompt = 4  # @param {type: "integer"}
batch_size = 4  # @param {type: "integer"}
clip_skip = 2  # @param {type: "slider", min: 1, max: 40}
seed = -1  # @param {type: "integer"}

final_prompt = f"{prompt} --n {negative}"

config = {
    "v2": v2,
    "v_parameterization": v_parameterization,
    "network_module": network_module,
    "network_weight": network_weight,
    "network_mul": float(network_mul),
    "network_args": eval(network_args) if network_args else None,
    "ckpt": model,
    "outdir": outdir,
    "xformers": True,
    "vae": vae if vae else None,
    "fp16": True,
    "W": width,
    "H": height,
    "seed": seed if seed > 0 else None,
    "scale": scale,
    "sampler": sampler,
    "steps": steps,
    "max_embeddings_multiples": 3,
    "batch_size": batch_size,
    "images_per_prompt": images_per_prompt,
    "clip_skip": clip_skip if not v2 else None,
    "prompt": final_prompt,
}

args = ""
for k, v in config.items():
    if k.startswith("_"):
        args += f'"{v}" '
    elif isinstance(v, str):
        args += f'--{k}="{v}" '
    elif isinstance(v, bool) and v:
        args += f"--{k} "
    elif isinstance(v, float) and not isinstance(v, bool):
        args += f"--{k}={v} "
    elif isinstance(v, int) and not isinstance(v, bool):
        args += f"--{k}={v} "

final_args = f"python gen_img_diffusers.py {args}"

# os.chdir(repo_dir)
!{final_args}